### Imports

In [0]:
import keras
from keras import backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.preprocessing.text import text_to_word_sequence, one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.stem.porter import *

import numpy as np
from numpy import array

import pandas as pd

from sklearn.utils import shuffle

### Upload data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving combined_data.csv to combined_data (3).csv


## Ignore

#### (I used this to preprocess and extract hate speech data from the other data sets and add them to the original csv file)

In [0]:
'''comment_frame = pd.read_csv("labeled_data.csv", error_bad_lines=False)             #io.StringIO(second_data['test_labels.csv'].decode('utf-8')))
#del comment_frame['id']
pd.set_option('display.max_colwidth', -1)
display(comment_frame)'''

'comment_frame = pd.read_csv("labeled_data.csv", error_bad_lines=False)             #io.StringIO(second_data[\'test_labels.csv\'].decode(\'utf-8\')))\n#del comment_frame[\'id\']\npd.set_option(\'display.max_colwidth\', -1)\ndisplay(comment_frame)'

In [0]:
'''
punctuation = "!\"#$%&()*+,'-./:;<=>?@[\]^_`{|}~"
table = str.maketrans("","",punctuation)


comments = comment_frame['comment_text']
#toxics = sorted_by_hate['toxic']
#severe_toxics = sorted_by_hate['severe_toxic']
#threats = sorted_by_hate['threat']
#insults = sorted_by_hate['insult']
hates = label_frame['identity_hate']

just_hate_comments = list()
just_hate_labels = list()
index = 0
for comment in comments:
    if hates[index] == 1:
        just_hate_comments.append("".join(comment.lower().translate(table).splitlines()))
        just_hate_labels.append(1)
    index += 1

#for entry in sorted_by_toxic:
    #print (entry)
    #if entry[2] == 0 and entry[3] == 0 and entry[4] == 0 and entry[5] == 0 and entry[6] == 0 and entry[7] == 0:
      #  print(entry)

#comments = data_frame['comment_text']
#print(comments)
#labels = data_frame['hate_speech']
#print(labels)
'''

'\npunctuation = "!"#$%&()*+,\'-./:;<=>?@[\\]^_`{|}~"\ntable = str.maketrans("","",punctuation)\n\n\ncomments = comment_frame[\'comment_text\']\n#toxics = sorted_by_hate[\'toxic\']\n#severe_toxics = sorted_by_hate[\'severe_toxic\']\n#threats = sorted_by_hate[\'threat\']\n#insults = sorted_by_hate[\'insult\']\nhates = label_frame[\'identity_hate\']\n\njust_hate_comments = list()\njust_hate_labels = list()\nindex = 0\nfor comment in comments:\n    if hates[index] == 1:\n        just_hate_comments.append("".join(comment.lower().translate(table).splitlines()))\n        just_hate_labels.append(1)\n    index += 1\n\n#for entry in sorted_by_toxic:\n    #print (entry)\n    #if entry[2] == 0 and entry[3] == 0 and entry[4] == 0 and entry[5] == 0 and entry[6] == 0 and entry[7] == 0:\n      #  print(entry)\n\n#comments = data_frame[\'comment_text\']\n#print(comments)\n#labels = data_frame[\'hate_speech\']\n#print(labels)\n'

In [0]:
'''

with open("kaggle_hate_test_comments.csv", "w") as kaggle_hate_test_comments:
    kaggle_hate_test_comments.write("comment,label\n")
    for comment in just_hate_comments:
        kaggle_hate_test_comments.write(str(comment) + ",1\n")

test_frame = pd.read_csv("kaggle_hate_test_comments.csv", error_bad_lines=False)
test_frame.columns
test_frame
'''

'\n\nwith open("kaggle_hate_test_comments.csv", "w") as kaggle_hate_test_comments:\n    kaggle_hate_test_comments.write("comment,label\n")\n    for comment in just_hate_comments:\n        kaggle_hate_test_comments.write(str(comment) + ",1\n")\n\ntest_frame = pd.read_csv("kaggle_hate_test_comments.csv", error_bad_lines=False)\ntest_frame.columns\ntest_frame\n'

## Start

In [0]:
data_frame = pd.read_csv(io.StringIO(uploaded['combined_data.csv'].decode('utf-8')))
data_frame.columns
data_frame

,comment,label
0,rt mayasolov as a woman you shouldnt complain ...,0
1,rt mleew17 boy dat coldtyga dwn bad for cuffin...,0
2,rt urkindofbrand dawg rt 80sbaby4lif you ever ...,0
3,rt cganderson vivabas she look like a tranni,0
4,rt shenikarobert the shit you hear about me mi...,0
...,...,...
25504,why do you support such vandals as peruvianlla...,1
25505,lol haha you loser wobbs yeah maybe i am epfno...,1
25506,usernhrhs2010 is a homo like mitt romney is us...,1
25507,fuck you barack obama is a muslim he sucks ass,1


In [0]:
#del data_frame['count']
#del data_frame['num']
#del data_frame['offensive_language']
#del data_frame['neither']
#data_frame

### Display Data

In [0]:
pd.set_option('display.max_colwidth', -1)
sorted_by_hate = data_frame.sort_values(by='label', axis='index', ascending=False)
display(sorted_by_hate)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,comment,label
25508,Does it surprise you that jews promote faggotry ?,1
16459,rt mikediggem onli ugli girl can fight pretti bitch have no hand lol,1
5272,zohdaa fag,1
5268,zhugstubbl naw nigga what up with em hand what happen to the shit you wa talk a coupl a month ago eat bitch,1
11634,if you enjoy hunt for sport then your trash if you hunt for food you deserv to starv,1
...,...,...
9703,have them hoe get old well it ha gotten old alreadi,0
9702,havent talk to my bitch in a while lol rynolewi,0
9700,have you hug a wackobird today,0
9699,have you heard 8216guala shit featur chasedadoeromeand the boy big8217 by joseph chase rinch on soundcloud httpstcooogna1zncr,0


### Extract Tweets

In [0]:
tweets = data_frame['comment']
print(tweets)
labels = data_frame['label']
print(labels)

0        rt mayasolov as a woman you shouldnt complain about clean up your hous amp as a man you should alway take the trash out                                                             
1        rt mleew17 boy dat coldtyga dwn bad for cuffin dat hoe in the 1st place                                                                                                             
2        rt urkindofbrand dawg rt 80sbaby4lif you ever fuck a bitch and she start to cri you be confus as shit                                                                               
3        rt cganderson vivabas she look like a tranni                                                                                                                                        
4        rt shenikarobert the shit you hear about me might be true or it might be faker than the bitch who told it to ya 57361                                                               
                                                  

# Preprocessing

## Preprocesses Tweets
- Remove punctuation, all lowercase, stemmer
- Count number of unique vocab words

## Preprocesses Labels
- Consider Hate speech if any worker assigned it as hate speech

### Calculate how many unique vocab words there are in the Tweets (answer: 36,280)

In [0]:
index = 10000
punctuation = "!\"#$%&()*+,'-./:;<=>?@[\]^_`{|}~"
table = str.maketrans("","",punctuation)
vocab = set()
processed_tweets = []

stemmer = PorterStemmer()

for tweet in tweets:
    processed_tweet = tweet.lower().translate(table).split()
    processed_tweet = [stemmer.stem(t) for t in processed_tweet]
    vocab = vocab.union(processed_tweet)
    processed_tweets.append(" ".join(processed_tweet))

vocab = sorted(vocab)
print(len(vocab))
num_vocab = len(vocab)

#binary_labels = []

#for label in labels:
#    if label > 0:
 #       binary_labels.append(1)
 #   else:
  #      binary_labels.append(0)
#
# Number of tweets for each label
# unique, counts = np.unique(labels.values, return_counts=True)
# dict(zip(unique, counts))

37535


In [0]:
#with open("combined_data.csv", "w") as combined_data:
#    combined_data.write("comment,label\n")
#    index = 0
#    for ptweet in processed_tweets:
#        combined_data.write(str(ptweet) + "," + str(binary_labels[index]) + "\n")
#        index += 1
#
#test_frame = pd.read_csv("combined_data.csv", error_bad_lines=False)
#test_frame.columns
#test_frame

In [0]:
print(vocab)

['0', '00', '0001', '007', '007beardownjedi', '007hertzrumbl', '007mh', '00jacki', '00sexilexi00', '01', '0108', '0124jessi', '01takesthelead', '02', '03', '0300', '05', '05235fd083ad408', '05professor', '06', '07cannon', '08', '09', '0bama', '0beythelau', '0beyyourmast', '0biwankob', '0ddment', '0hallis0n', '0kkaren', '0lt', '0o', '0oitscrunch', '0utcast', '0xabad1dea', '1', '10', '100', '1000', '10000', '100000', '1000465039', '10004650398230', '100046729', '1000gramsb', '1000x', '1005265039127744', '10060', '100601109365039100601109365039127801', '10060128282', '100671285531285141285148221', '1007165039', '1007165039100698252650391006910069', '10084', '100841008410084100841008410084100848221', '10084128076', '1008465039', '1008465039128076', '1008465039128154', '1008465039128536', '10084650398221', '100bandrel', '100d', '100gram', '100granhman', '100k', '100x', '1010', '1017', '1017sqquad', '1022pm', '102312', '1028', '103', '1030', '1040784999665039', '10411841041185', '10411841043

Calculate max len (answer: 33 )

In [0]:
def fa(x):
  return len(x.split())
maxlen = np.max(np.vectorize(fa)(processed_tweets))
maxlen
#maxlen = 990

354

## Hyperparameters

In [0]:
#### input_dim=50000 is the maximum number of unique vocab words
#### input_length=33 is the max length of the padded Tweets
### output_dim=512 is a hyperparameter -- you can make it anything

# Embedding
max_features = 1000 #37599 #1000 having less is better because what this means is less frequent words will be removed ; num_vocab #37599
maxlen = maxlen # = 990
embedding_size = 100 #was 128

# Convolution
kernel_size = 5 # was 3
filters = 64 #16 
pool_size = 4

# LSTM
lstm_output_size =128 # 64   16 

# Training
batch_size = 32 #was 36
epochs = 3 #was 2

## Tokenize & Pad Tweets

In [0]:
tweets, labels = shuffle(tweets, labels)

x_train = tweets[:20000]
x_test = tweets[20000:]

y_train = labels[:20000]
y_test = labels[20000:]

tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_matrix(x_train)
x_test = tokenizer.texts_to_matrix(x_test)

x_train = pad_sequences(x_train, maxlen= maxlen) #dtype='int32', padding='post', value=0
x_test = pad_sequences(x_test, maxlen= maxlen)


#encoded_tweets = [one_hot(tweet, num_vocab) for tweet in processed_tweets]
#padded_tweets = pad_sequences(encoded_tweets, dtype='int32', padding='post', value=0, maxlen= maxlen)
#print(padded_tweets)

## Use GloVe embeddings

#### Acquire GloVe file

In [133]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2020-04-19 19:00:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-04-19 19:00:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-04-19 19:00:06--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [134]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.200d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
# Use GloVe embeddings:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((num_vocab, 100))
for word, index in tokenizer.word_index.items():
    if index > num_vocab - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

## Build model

In [0]:
model = Sequential()
model.add(Embedding(num_vocab, embedding_size, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Dropout(0.5)) #0.25
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu')) #,
                 #strides=2))
model.add(MaxPooling1D()) #(pool_size=pool_size))
model.add(LSTM(lstm_output_size, activation="sigmoid"))
model.add(Dense(1, activation="sigmoid"))
#model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [0]:
#import numpy as np
#np.reshape(tweets, (-1,1))
#processed_tweets.transpose().shape

#processed_tweets = processed_tweets.reshape((24783, 1))
#keras.preprocessing.sequence.pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)

# Train and test model

In [162]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test Loss:', score)
print('Test accuracy:' + str(acc*100) + "%")

Train...
Train on 20000 samples, validate on 5509 samples
Epoch 1/4
20000/20000 [==============================] - 178s 9ms/step - loss: 0.5352 - accuracy: 0.7754 - val_loss: 0.5461 - val_accuracy: 0.7771
Epoch 2/4
20000/20000 [==============================] - 176s 9ms/step - loss: 0.5343 - accuracy: 0.7754 - val_loss: 0.5309 - val_accuracy: 0.7771
Epoch 3/4
20000/20000 [==============================] - 175s 9ms/step - loss: 0.5342 - accuracy: 0.7754 - val_loss: 0.5310 - val_accuracy: 0.7771
Epoch 4/4
5509/5509 [==============================] - 4s 730us/step
Test Loss: 0.531163770688556
Test accuracy:77.70920395851135%


# Save Model

In [0]:
model.save('hate_speech_model.h5')

#### Load model

In [0]:
#from keras.models import load_model
#new_model = load_model('hate_speech_model.h5')